In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path


In [2]:
root = Path("/kaggle/input/mabe-merged-data")
for dir in root.rglob("*"):
    if dir.is_dir():
        print(dir)


/kaggle/input/mabe-merged-data/kaggle
/kaggle/input/mabe-merged-data/kaggle/working
/kaggle/input/mabe-merged-data/kaggle/working/tracking
/kaggle/input/mabe-merged-data/kaggle/working/annotated


In [3]:
afp = Path("/kaggle/input/mabe-merged-data/kaggle/working/annotated/1009459450.parquet")
tfp = Path("/kaggle/input/mabe-merged-data/kaggle/working/tracking/1000217804.parquet")
dfa = pd.read_parquet(afp)
dft = pd.read_parquet(tfp)

In [4]:
print(dft.head())
print(dft.info())

  lateral_left_y tail_middle_1_y  hindpaw_right_x lateral_left_x  \
0           None            None            380.0           None   
1           None            None            171.0           None   
2           None            None            172.0           None   
3           None            None            381.0           None   
4           None            None            171.0           None   

   body_center_x headpiece_bottombackleft_x spine_1_x hip_right_y  \
0          380.0                       None      None        None   
1          157.0                       None      None        None   
2          170.0                       None      None        None   
3          380.0                       None      None        None   
4          157.0                       None      None        None   

  tail_middle_2_x  ear_left_x  ... spine_2_y headpiece_bottomfrontleft_y  \
0            None       352.0  ...      None                        None   
1            None       

In [5]:
print(afp.resolve())
print(dfa.head())
print(dfa.info())

/kaggle/input/mabe-merged-data/kaggle/working/annotated/1009459450.parquet
  lateral_left_y tail_middle_1_y hindpaw_right_x lateral_left_x body_center_x  \
0           None            None            None           None          None   
1           None            None            None           None          None   
2           None            None            None           None          None   
3           None            None            None           None          None   
4           None            None            None           None          None   

  headpiece_bottombackleft_x spine_1_x  hip_right_y tail_middle_2_x  \
0                       None      None   370.541107            None   
1                       None      None   369.221863            None   
2                       None      None   369.698364            None   
3                       None      None   363.496368            None   
4                       None      None   370.619690            None   

   ear_left

In [6]:
# Define the folder path
comp_afp = Path("/kaggle/input/MABe-mouse-behavior-detection/train_annotation")

# Collect all parquet files matching the pattern
files = list(comp_afp.rglob("1009459450.parquet"))

# Make sure at least one file was found
print(f"Found {len(files)} files")

# Load the first one
if files:
    actual_file_data = pd.read_parquet(files[0])
    print(actual_file_data.info())
    print(actual_file_data.head())
else:
    print("No parquet files found matching that name.")

Found 1 files
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   agent_id     126 non-null    int8  
 1   target_id    126 non-null    int8  
 2   action       126 non-null    object
 3   start_frame  126 non-null    int16 
 4   stop_frame   126 non-null    int16 
dtypes: int16(2), int8(2), object(1)
memory usage: 1.9+ KB
None
   agent_id  target_id    action  start_frame  stop_frame
0         1          2  approach          964         991
1         1          2     sniff          992        1130
2         1          2     sniff         1155        1165
3         1          2     sniff         1229        1275
4         1          2     sniff         1287        1381


In [7]:
# check = Path("/kaggle/working/annotated_fixed/1009459450_fixed.parquet")
# check_df=pd.read_parquet(check)
# print(check_df.head())
# print(check_df.info())

In [8]:
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm  # Progress bar for notebooks

broken_dir = Path("/kaggle/input/mabe-merged-data/kaggle/working/annotated")
anno_dir = Path("/kaggle/input/MABe-mouse-behavior-detection/train_annotation")
output_dir = Path("/kaggle/working/annotated_fixed")
output_dir.mkdir(exist_ok=True, parents=True)

def expand_annotations_vectorized(anno_df):
    """
    Expand start/stop frame intervals to per-frame rows.
    Keeps agent_id separate from mouse_id to avoid overwriting coordinates owner.
    """
    lengths = anno_df['stop_frame'] - anno_df['start_frame'] + 1
    frames = np.concatenate([np.arange(start, stop + 1) for start, stop in zip(anno_df['start_frame'], anno_df['stop_frame'])])
    
    agent_ids = np.repeat(anno_df['agent_id'].values, lengths)
    target_ids = np.repeat(anno_df['target_id'].values, lengths)
    actions = np.repeat(anno_df['action'].values, lengths)
    
    return pd.DataFrame({
        'video_frame': frames,
        'agent_id': agent_ids,       # mouse performing the action
        'target_id': target_ids,     # target of the action
        'action': actions
    })

# Build a dictionary mapping video_id -> annotation file path (recursively)
anno_files = {f.stem: f for f in anno_dir.rglob("*.parquet")}

# Loop over all broken merged files with a progress bar
for broken_file in tqdm(list(broken_dir.glob("*.parquet")), desc="Fixing files"):
    vid_id = broken_file.stem

    # Skip if fixed file already exists
    out_path = output_dir / f"{vid_id}_fixed.parquet"
    if out_path.exists():
        continue

    # Skip if there is no broken file to fix
    if vid_id not in anno_files:
        continue

    anno_file = anno_files[vid_id]

    try:
        # Load merged tracking data and annotation file
        merged_df = pd.read_parquet(broken_file)
        anno_df = pd.read_parquet(anno_file)

        # Expand annotations to per-frame rows
        anno_expanded = expand_annotations_vectorized(anno_df)

        # Merge annotations on video_frame only, preserving mouse_id
        fixed_df = merged_df.drop(columns=["action", "target_id", "agent_id"], errors="ignore").merge(
            anno_expanded, on="video_frame", how="left"
        )

        # Save fixed file
        out_path = output_dir / f"{vid_id}_fixed.parquet"
        fixed_df.to_parquet(out_path, index=False)

    except Exception as e:
        print(f"Error processing {vid_id}: {e}")

print("🎉 All done! Check /kaggle/working/annotated_fixed for repaired files.")


Fixing files:   0%|          | 0/847 [00:00<?, ?it/s]

🎉 All done! Check /kaggle/working/annotated_fixed for repaired files.


In [9]:
df = pd.read_parquet("/kaggle/input/fixed-merged-files/annotated_fixed/1009459450_fixed.parquet")
print(df.head())
print(df.info())


  lateral_left_y tail_middle_1_y hindpaw_right_x lateral_left_x body_center_x  \
0           None            None            None           None          None   
1           None            None            None           None          None   
2           None            None            None           None          None   
3           None            None            None           None          None   
4           None            None            None           None          None   

  headpiece_bottombackleft_x spine_1_x  hip_right_y tail_middle_2_x  \
0                       None      None   370.541107            None   
1                       None      None   369.221863            None   
2                       None      None   369.698364            None   
3                       None      None   363.496368            None   
4                       None      None   370.619690            None   

   ear_left_x  ... headpiece_bottomfrontright_y mouse_id  forepaw_right_x  \
0  285.13

In [10]:
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm

# Directories
data_dir = Path("/kaggle/working/annotated_fixed")  # input fixed merged files
output_dir = Path("/kaggle/working/annotated_core")  # output cleaned files
output_dir.mkdir(exist_ok=True, parents=True)

files = list(data_dir.glob("*.parquet"))

# Step 1: Identify core body parts
core_columns = None

for f in tqdm(files, desc="Finding core body parts"):
    # If the cleaned data file exists, skip it
    out_path = output_dir / f.name
    if out_path.exists():
        continue
    df = pd.read_parquet(f)
    coord_cols = [c for c in df.columns if c not in ['video_frame', 'mouse_id', 'agent_id', 'target_id', 'action']]
    tracked_cols = [c for c in coord_cols if df[c].notna().any()]
    
    if core_columns is None:
        core_columns = set(tracked_cols)
        continue
    else:
        core_columns &= set(tracked_cols)

core_columns = sorted(core_columns)
print(f"✅ Core tracked body parts ({len(core_columns)}): {core_columns}")

# Step 2: Save updated files with only core features
for f in tqdm(files, desc="Saving cleaned parquet files"):
    df = pd.read_parquet(f)
    df_core = df[core_columns + ['mouse_id', 'video_frame', 'agent_id', 'target_id', 'action']]
    
    out_path = output_dir / f.name
    df_core.to_parquet(out_path, index=False)

print(f"🎉 All done! Cleaned files are in: {output_dir}")


Finding core body parts:   0%|          | 0/847 [00:00<?, ?it/s]

✅ Core tracked body parts (6): ['ear_left_x', 'ear_left_y', 'ear_right_x', 'ear_right_y', 'tail_base_x', 'tail_base_y']


Saving cleaned parquet files:   0%|          | 0/847 [00:00<?, ?it/s]

🎉 All done! Cleaned files are in: /kaggle/working/annotated_core


In [11]:
from pathlib import Path
import pandas as pd

core_dir = Path("/kaggle/working/annotated_core")
files = list(core_dir.glob("*.parquet"))

# Collect columns from all files
all_columns = []

for f in files:
    df = pd.read_parquet(f)
    all_columns.append(tuple(df.columns))  # use tuple to make it hashable

# Check if all column sets are identical
unique_column_sets = set(all_columns)

if len(unique_column_sets) == 1:
    print("✅ All files have consistent columns.")
    print("Columns:", list(unique_column_sets.pop()))
else:
    print(f"⚠️ Found {len(unique_column_sets)} different column sets across files.")
    for cols in unique_column_sets:
        print(cols)


✅ All files have consistent columns.
Columns: ['ear_left_x', 'ear_left_y', 'ear_right_x', 'ear_right_y', 'tail_base_x', 'tail_base_y', 'mouse_id', 'video_frame', 'agent_id', 'target_id', 'action']


In [12]:
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime

# --- Paths ---
tracking_dir = Path("/kaggle/input/mabe-merged-data/kaggle/working/tracking")
output_tracking_dir = Path("/kaggle/working/tracking_core")
summary_dir = output_tracking_dir / "summary"
summary_dir.mkdir(exist_ok=True, parents=True)

tracking_files = list(tracking_dir.glob("*.parquet"))

# --- Step 1: Identify core body parts in tracking files ---
core_columns = None
total_columns_before = 0
total_columns_after = 0

for f in tqdm(tracking_files, desc="Finding core tracking body parts"):
    out_path = output_tracking_dir / f.name
    if out_path.exists():
        continue

    df = pd.read_parquet(f)
    total_columns_before += len(df.columns)

    # Exclude ID/annotation columns if present
    exclude_cols = ['video_frame','mouse_id','agent_id', 'target_id', 'action']
    coord_cols = [c for c in df.columns if c not in exclude_cols]

    tracked_cols = [c for c in coord_cols if df[c].notna().any()]

    if core_columns is None:
        core_columns = set(tracked_cols)
    else:
        core_columns &= set(tracked_cols)

core_columns = sorted(core_columns)
print(f"✅ Core tracked body parts ({len(core_columns)}): {core_columns}")

# --- Step 2: Save cleaned tracking files with only core features ---
for f in tqdm(tracking_files, desc="Saving cleaned tracking files"):
    df = pd.read_parquet(f)
    
    # Keep only core body parts + essential IDs if present
    keep_cols = core_columns.copy()
    for col in ['mouse_id', 'video_frame', 'agent_id', 'target_id', 'action']:
        if col in df.columns:
            keep_cols.append(col)

    df_core = df[keep_cols]
    total_columns_after += len(df_core.columns)

    out_path = output_tracking_dir / f.name
    df_core.to_parquet(out_path, index=False)

print(f"🎉 All done! Cleaned tracking files are in: {output_tracking_dir}")

# --- Step 3: Calculate aggregated column removal stats ---
total_columns_dropped = total_columns_before - total_columns_after
avg_columns_dropped = total_columns_dropped / len(tracking_files)

# --- Step 4: Create summary Markdown file ---
summary_path = summary_dir / "summary.md"
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

summary_text = f"""# Tracking Core Cleaning Summary
**Generated:** {timestamp}

## Overview
This process created cleaned tracking files based on the intersection of body part coordinates 
that were consistently tracked across all input `.parquet` files.

### Input Directory
`{tracking_dir}`

### Output Directory
`{output_tracking_dir}`

### Total Files Processed
{len(tracking_files)}

### Core Body Parts Retained
{len(core_columns)} total columns retained:
{', '.join(core_columns)}

### ID Columns Preserved
The following identifier columns were added back if initially present:
- `mouse_id`
- `video_frame`
- `agent_id`
- `target_id`
- `action`

### Column Reduction Summary
- **Total columns removed (aggregated across all files):** {total_columns_dropped}
- **Average columns removed per file:** {avg_columns_dropped:.2f}

### Notes
- Files were saved in the output directory with only the above columns retained.
- Original input files remain unchanged.
"""

with open(summary_path, "w") as f:
    f.write(summary_text)

print(f"📝 Summary saved to: {summary_path}")


Finding core tracking body parts:   0%|          | 0/7942 [00:00<?, ?it/s]

✅ Core tracked body parts (10): ['body_center_x', 'body_center_y', 'ear_left_x', 'ear_left_y', 'ear_right_x', 'ear_right_y', 'nose_x', 'nose_y', 'tail_base_x', 'tail_base_y']


Saving cleaned tracking files:   0%|          | 0/7942 [00:00<?, ?it/s]

🎉 All done! Cleaned tracking files are in: /kaggle/working/tracking_core
📝 Summary saved to: /kaggle/working/tracking_core/summary/summary.md


In [13]:
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter
from datetime import datetime
import shutil
import os

# --- Paths ---
tracking_core_dir = Path("/kaggle/working/tracking_core")
missing_info_dir = Path("/kaggle/working/tracking_core_missing_info")
summary_dir = tracking_core_dir / "summary"
summary_dir.mkdir(exist_ok=True, parents=True)
missing_info_dir.mkdir(exist_ok=True, parents=True)

files = list(tracking_core_dir.glob("*.parquet"))

# --- Check column consistency ---
all_columns = [tuple(pd.read_parquet(f).columns) for f in tqdm(files, desc="Checking column sets")]
unique_column_sets = set(all_columns)

if len(unique_column_sets) == 1:
    print("✅ All files have consistent columns.")
    column_list = list(unique_column_sets.pop())
else:
    print(f"⚠️ Found {len(unique_column_sets)} different column sets across files.")
    column_list = list(list(unique_column_sets)[0])

# --- Check for NaNs in core body parts ---
core_columns = [c for c in column_list]

nan_file_counts = {}        # {filename: total NaN count}
nan_column_occurrences = Counter()  # counts how many files each column has NaNs
moved_files = []

for f in tqdm(files, desc="Checking for NaNs"):
    df = pd.read_parquet(f)
    nan_per_col = df[core_columns].isna().sum()
    total_nans = nan_per_col.sum()
    
    if total_nans > 0:
        nan_file_counts[f.name] = int(total_nans)
        nan_columns = nan_per_col[nan_per_col > 0].index.tolist()
        nan_column_occurrences.update(nan_columns)

        # --- Move file safely ---
        dest_path = missing_info_dir / f.name

        # Skip if already in destination
        if not dest_path.exists():
            shutil.move(str(f), str(dest_path))
            moved_files.append(f.name)
        else:
            # If destination already has file, delete the source file
            try:
                os.remove(f)
                print(f"⚠️ Deleted duplicate source file: {f.name}")
            except Exception as e:
                print(f"⚠️ Error deleting duplicate {f.name}: {e}")

# --- Summarize results ---
summary_lines = []
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

if not nan_file_counts:
    print("✅ No NaNs found in core body parts across all files.")
    summary_lines.append("# NaN Analysis Summary\n")
    summary_lines.append(f"**Generated:** {timestamp}\n\n")
    summary_lines.append("✅ No NaNs found in core body parts across all files.\n")
else:
    total_nan_files = len(nan_file_counts)
    total_nans_overall = sum(nan_file_counts.values())
    print(f"⚠️ Found NaNs in {total_nan_files} files.")
    print(f"📦 Moved {len(moved_files)} new files with NaNs to: {missing_info_dir}")

    summary_lines.append("# NaN Analysis Summary\n")
    summary_lines.append(f"**Generated:** {timestamp}\n\n")
    summary_lines.append(f"Found NaNs in **{total_nan_files}** out of **{len(files)}** files.\n")
    summary_lines.append(f"Moved {len(moved_files)} new files with NaNs to `{missing_info_dir}`.\n")
    summary_lines.append(f"Total NaN values across all files: {total_nans_overall}\n")

    # --- Top 3 most common NaN totals ---
    top_nan_counts = Counter(nan_file_counts.values()).most_common(3)
    summary_lines.append("\n## Top 3 Most Common Total NaN Counts\n")
    for count, freq in top_nan_counts:
        summary_lines.append(f"- {count} NaNs — {freq} files\n")

    # --- Columns most frequently containing NaNs ---
    summary_lines.append("\n## Columns Most Frequently Containing NaNs\n")
    for col, freq in nan_column_occurrences.most_common(10):
        summary_lines.append(f"- {col}: NaNs found in {freq} files\n")

    # --- Moved file list (first 10 only) ---
    summary_lines.append("\n## Sample of Moved Files (first 10)\n")
    for fname in moved_files[:10]:
        summary_lines.append(f"- {fname}\n")

# --- Save summary file ---
summary_path = summary_dir / "nan_summary.md"
with open(summary_path, "w") as f:
    f.writelines(summary_lines)

print(f"\n📝 Summary saved to: {summary_path}")


Checking column sets:   0%|          | 0/7942 [00:00<?, ?it/s]

✅ All files have consistent columns.


Checking for NaNs:   0%|          | 0/7942 [00:00<?, ?it/s]

⚠️ Found NaNs in 7942 files.
📦 Moved 7942 new files with NaNs to: /kaggle/working/tracking_core_missing_info

📝 Summary saved to: /kaggle/working/tracking_core/summary/nan_summary.md


In [14]:
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter, defaultdict
from datetime import datetime

# --- Paths ---
tracking_core_dir = Path("/kaggle/input/fixed-merged-files/tracking_core")
files = list(tracking_core_dir.glob("*.parquet"))

summary_dir = Path("/kaggle/working/summary")
summary_dir.mkdir(parents=True, exist_ok=True)
summary_path = summary_dir / "nan_summary.md"

# --- Check column consistency ---
all_columns = [tuple(pd.read_parquet(f).columns) for f in tqdm(files, desc="Checking column sets")]
unique_column_sets = set(all_columns)

if len(unique_column_sets) == 1:
    print("✅ All files have consistent columns.")
    column_list = list(unique_column_sets.pop())
else:
    print(f"⚠️ Found {len(unique_column_sets)} different column sets across files.")
    column_list = list(list(unique_column_sets)[0])

# --- Exclude metadata columns from NaN check ---
exclude_cols = {"agent_id", "target_id", "action"}

# --- Initialize tracking containers ---
core_columns = [c for c in column_list if c not in exclude_cols]
nan_counts = {}
nan_columns = {}
video_frame_counts = {}
mouse_id_counts = {}

# For column-level NaN stats
nan_column_occurrences = Counter()     # counts how many files each column has NaNs
nan_column_totals = defaultdict(int)   # counts total NaN instances per column across all files

# --- Analyze files ---
for f in tqdm(files, desc="Analyzing files"):
    df = pd.read_parquet(f)

    # NaN check: total and per-column (excluding metadata columns)
    nan_per_col = df[core_columns].isna().sum()
    total_nans = nan_per_col.sum()
    nan_counts[f.name] = total_nans

    nan_cols = nan_per_col[nan_per_col > 0]
    nan_columns[f.name] = nan_cols.to_dict()

    # Update per-column aggregate stats
    for col, count in nan_cols.items():
        nan_column_occurrences[col] += 1
        nan_column_totals[col] += int(count)

    # Unique IDs
    video_frame_counts[f.name] = df["video_frame"].nunique() if "video_frame" in df.columns else None
    mouse_id_counts[f.name] = df["mouse_id"].value_counts().to_dict() if "mouse_id" in df.columns else {}

# --- NaN summary ---
files_with_nans = {k: v for k, v in nan_counts.items() if v > 0}

# --- Aggregate mouse_id counts across all files ---
all_mouse_ids = {}
for mouse_dict in mouse_id_counts.values():
    for mid, count in mouse_dict.items():
        all_mouse_ids[mid] = all_mouse_ids.get(mid, 0) + count

# --- Build Markdown Summary ---
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
summary_lines = [
    "# NaN Analysis Summary\n",
    f"**Generated:** {timestamp}\n\n",
    f"**Input directory:** `{tracking_core_dir}`\n\n",
    f"**Total files analyzed:** {len(files)}\n\n",
    f"**Excluded columns from NaN check:** {', '.join(exclude_cols)}\n\n"
]

if not files_with_nans:
    print("✅ No NaNs found in core body parts across all files.")
    summary_lines.append("✅ No NaNs found in core body parts across all files.\n")
else:
    total_nan_files = len(files_with_nans)
    total_nans_overall = int(sum(files_with_nans.values()))

    print(f"⚠️ Found NaNs in {total_nan_files} files.")
    summary_lines.append(f"Found NaNs in **{total_nan_files}** out of **{len(files)}** files.\n")
    summary_lines.append(f"Total NaN values across all files: **{total_nans_overall}**\n")

    # --- Top 3 most common total NaN counts ---
    top_nan_counts = Counter(files_with_nans.values()).most_common(3)
    summary_lines.append("\n## Top 3 Most Common Total NaN Counts\n")
    for count, freq in top_nan_counts:
        summary_lines.append(f"- {count} NaNs — {freq} files\n")

    # --- Columns most frequently containing NaNs ---
    summary_lines.append("\n## Columns Most Frequently Containing NaNs\n")
    summary_lines.append("*(Sorted by number of files with NaNs, then total NaN count)*\n")
    sorted_columns = sorted(
        nan_column_occurrences.items(),
        key=lambda x: (x[1], nan_column_totals[x[0]]),
        reverse=True
    )
    for col, freq in sorted_columns[:10]:
        total_for_col = nan_column_totals[col]
        summary_lines.append(f"- {col}: NaNs in {freq} files (total {total_for_col} missing entries)\n")

# --- Add Unique ID Summary ---
summary_lines.append("\n## Unique ID Summary\n")
summary_lines.append(f"- Total unique `mouse_id` values: {len(all_mouse_ids)}\n")
summary_lines.append("- Top 10 most frequent mouse_ids:\n")

mouse_summary = pd.Series(all_mouse_ids).sort_values(ascending=False).head(10)
for idx, val in mouse_summary.items():
    summary_lines.append(f"  - {idx}: {val}\n")

# --- Save Markdown Summary ---
with open(summary_path, "w") as f:
    f.writelines(summary_lines)

print(f"\n📝 Summary saved to: {summary_path}")


Checking column sets:   0%|          | 0/7942 [00:00<?, ?it/s]

✅ All files have consistent columns.


Analyzing files:   0%|          | 0/7942 [00:00<?, ?it/s]

⚠️ Found NaNs in 1 files.

📝 Summary saved to: /kaggle/working/summary/nan_summary.md


In [15]:
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm  # or just `from tqdm import tqdm` outside Jupyter
from datetime import datetime

# --- Define paths ---
tracking_core_dir = Path("/kaggle/input/fixed-merged-files/tracking_core")
output_tracking_core_dir = Path("/kaggle/working/tracking_core")
summary_dir = output_tracking_core_dir / "summary"
summary_dir.mkdir(parents=True, exist_ok=True)
summary_path = summary_dir / "tracking_core_cleanup_summary.md"

output_tracking_core_dir.mkdir(parents=True, exist_ok=True)

# --- List parquet files ---
files = list(tracking_core_dir.glob("*.parquet"))
print(f"Found {len(files)} tracking files.")

# --- Columns to remove ---
cols_to_remove = ["agent_id", "action", "target_id"]

# --- Counters for removals ---
remove_counts = {col: 0 for col in cols_to_remove}
total_files_processed = 0

# --- Process files ---
for f in tqdm(files, desc="Cleaning files"):
    df = pd.read_parquet(f)
    total_files_processed += 1

    # Check which columns exist and should be removed
    drop_cols = [c for c in cols_to_remove if c in df.columns]
    for c in drop_cols:
        remove_counts[c] += 1

    # Drop them if present
    if drop_cols:
        df = df.drop(columns=drop_cols)
    
    # Save cleaned file
    out_path = output_tracking_core_dir / f.name
    df.to_parquet(out_path, index=False)

# --- Summary output ---
print("\nDone! Cleaned files saved to:", output_tracking_core_dir)
print("Column removal summary:")
for col, count in remove_counts.items():
    print(f"   {col}: removed from {count} files")

# --- Create summary Markdown file ---
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

summary_lines = [
    "# Tracking Core Cleanup Summary\n",
    f"**Generated:** {timestamp}\n\n",
    f"**Input directory:** `{tracking_core_dir}`\n\n",
    f"**Output directory:** `{output_tracking_core_dir}`\n\n",
    f"**Total files processed:** {total_files_processed}\n\n",
    "## Columns Removed\n",
]

total_removed_any = 0
for col, count in remove_counts.items():
    summary_lines.append(f"- `{col}`: removed from **{count}** files\n")
    total_removed_any += count

summary_lines.append(f"\n**Total removals across all columns:** {total_removed_any}\n")

summary_lines.append("\n### Notes\n")
summary_lines.append(
    "- Only the specified columns (`agent_id`, `action`, `target_id`) were targeted for removal.\n"
)
summary_lines.append(
    "- Files that did not contain these columns were left unchanged.\n"
)
summary_lines.append(
    "- Each cleaned file retains all other original data and structure.\n"
)

# --- Save Markdown summary ---
with open(summary_path, "w") as f:
    f.writelines(summary_lines)

print(f"\n📝 Summary saved to: {summary_path}")


Found 7942 tracking files.


Cleaning files:   0%|          | 0/7942 [00:00<?, ?it/s]


Done! Cleaned files saved to: /kaggle/working/tracking_core
Column removal summary:
   agent_id: removed from 7942 files
   action: removed from 7942 files
   target_id: removed from 7942 files

📝 Summary saved to: /kaggle/working/tracking_core/summary/tracking_core_cleanup_summary.md
